In [ ]:
import torch
import python_speech_features

import librosa
import librosa.display

import os
from matplotlib import pyplot as plt
import numpy as np

## Text2speech

https://github.com/snakers4/silero-models#text-to-speech

In [ ]:
device = torch.device('cuda')
#torch.set_num_threads(4)
local_file = 'tts.pt'

if not os.path.isfile(local_file):
    torch.hub.download_url_to_file('https://models.silero.ai/models/tts/ru/v2_kseniya.pt',
                                   local_file)  

model = torch.package.PackageImporter(local_file).load_pickle("tts_models", "model")
model.to(device)


In [ ]:
example_batch = [' В начале сотворил Бог небо и землю. Земля же была безвидна и пуста, и тьма над бездною, и Дух Божий носился над водою.',
                 ' И сказал Бог - да будет свет. И стал свет. И увидел Бог свет, что он хорош, и отделил Бог свет от тьмы.',
                 ' И назвал Бог свет днем, а тьму ночью. И был вечер, и было утро - день один. ',
                 ' И сказал Бог - да будет твердь посреди воды, и да отделяет она воду от воды',
                 'И создал Бог твердь, и отделил воду, которая под твердью, от воды, которая над твердью. И стало так.']
sample_rate = 16000


In [ ]:
audio_paths = model.save_wav(texts=example_batch,
                             sample_rate=sample_rate,
                                        )
print(audio_paths)

In [ ]:
wav=model.apply_tts(texts=example_batch)


In [ ]:
audio=wav[0].cpu().detach().numpy()
mfcc_data = python_speech_features.mfcc(audio, sample_rate * 2, nfft=1103)

In [ ]:
plt.plot(audio)

In [ ]:
spec = np.abs(librosa.stft(audio, hop_length=512))
spec = librosa.amplitude_to_db(spec, ref=np.max)
sr=16000
librosa.display.specshow(spec, sr=sr, x_axis='time', y_axis='log');
plt.colorbar(format='%+2.0f dB');
plt.title('Spectrogram');


In [ ]:
mfcc_data.shape

In [ ]:
model.write_wave('audio.wav',wav[0].cpu().detach().numpy(),16000)

## Speech2text

https://github.com/alphacep/vosk-api

In [2]:
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
from scipy.io import wavfile
import ctypes

In [3]:
stt_model_path='models/stt'

if not os.path.exists(stt_model_path):
    print ("Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in the current folder.")

else:
    stt_model = Model(stt_model_path)

In [48]:
speech_file='test_000.wav'
wf = wave.open(speech_file, "rb")

In [49]:
%%time
rec = KaldiRecognizer(stt_model, wf.getframerate())
data = wf.readframes(4000000)
rec.AcceptWaveform(data)
print(rec.FinalResult())

{
  "text" : "в начале сотворил бог небо и землю земля же была безвидна и пуста и тьма над бездною и дух божий носился над водою"
}
Wall time: 1.16 s
